# Compare Beams on Sky and Focal Plane
In this notebook, we compare the beam patterns on sky and focal plane and show that they are identical.

Chun Tung Cheung (March 2022)

In [1]:
import numpy as np
import sys
import os
import pickle

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from DEFAULTS import PARENT_PATH
import sim.ap_field_ar as ap
import sim.far_field_ar as ff
import sim.tele_geo_ar as tg
import sim.pan_mod_ar as pm

from utility.conversion import sky_to_focal, flat_sqgrid_to_ticks, flat_sqgrid_to_2dgrid
from utility.plotting import scalar_field_plot

## Read the simulation data of the focal plane beam and the panel errors.

In [2]:
# load simulation data of focal plane beam
# sim_file = "E:\\Holography\\lat_holocal\\data\\ctcheung\\2022_02_04_04_05\\simdata_2.pys"
sim_file = PARENT_PATH + "\\dummydata\\focal_field_35.pys"
# dataset = dict(adj_err_1=[], adj_err_2=[], field_fp=[])
dataset = {}
with open(sim_file, "rb") as file:
    sim_data = pickle.load(file)
dataset["adj_err_1"] = sim_data["adj_err_1"]
dataset["adj_err_2"] = sim_data["adj_err_2"]
dataset["field_fp"] = sim_data["field_fp"]

# create panel offsets randomly
save = 0 
adj_1_A = dataset["adj_err_1"]
adj_2_A = dataset["adj_err_2"]
panel_model2 = pm.panel_model_from_adjuster_offsets(
    2, adj_2_A, 1, save
)  # Panel Model on M2
panel_model1 = pm.panel_model_from_adjuster_offsets(
    1, adj_1_A, 1, save
)  # Panel Model on M1

## Simulate the farfield beam pattern on sky using the loaded panel errors

In [3]:
# ###################### define telescope geometry
freq = 101.28 # frequency of signal source [GHz]
wavelength = (30.0 / freq) * 0.01 # [m]
wavelength = wavelength * 1e3 # [mm]
k = 2 * np.pi / wavelength
# positions of the source and receiver feedhorns [mm]
P_source = np.array([0, -7200.0, 1e6]) # unit [mm]
P_rx = np.array([0, 209.920654, -1.36822040e-02]) # unit [mm]
# P_source = np.array([0, -7200.0, 1e12]) # unit [mm]
# P_rx = np.array([0, 0, 0])
st_th_fwhp = 30.0/180.0*np.pi 

tele_geo = tg.TelescopeGeometry()
tele_geo.N_scan = 50  # pixels in 1D of grid
# tele_geo.de_ang = 1.0 / 60.0 * np.pi / 180.0  # Far-field angle increment, arcmin = 1/60 degree
arcmin_to_rad = 1.0 / 60 * np.pi / 180.0
tele_geo.de_ang = 60.0/tele_geo.N_scan / (252.0/60.0) * arcmin_to_rad   # angle increment of telescope scan [rad]

tele_geo.lambda_ = wavelength / 1e3 # [m]
tele_geo.k = 2 * np.pi / tele_geo.lambda_  # wavenumber [1/m]
# tele_geo.th_fwhp = 44 * np.pi / 180  # full width half power of source feed [rad]
[tele_geo.x_tow, tele_geo.y_tow, tele_geo.z_tow] = P_source/1e3
[tele_geo.rx_x, tele_geo.rx_y, tele_geo.rx_z] = P_rx/1e3

# Azimuth and Elevation center [rad]
tele_geo.az0 = 0.0
tele_geo.el0 = np.arctan(-tele_geo.y_tow / tele_geo.z_tow)
# tele_geo.el0 = 7.6/1e3
# tele_geo.el0 = 0.0

# arrays of pointing angles of rays
angle_size = 0.29
theta_a, theta_b, theta_N = -np.pi / 2 - angle_size, -np.pi / 2 + angle_size, 100
phi_a, phi_b, phi_N = np.pi / 2 - angle_size, np.pi / 2 + angle_size, 100
theta = np.linspace(theta_a, theta_b, theta_N)
phi = np.linspace(phi_a, phi_b, phi_N)



# get parameters from telescope geometry
tg_th_1, tg_th2, tg_z_ap = tele_geo.th_1, tele_geo.th2, tele_geo.z_ap
tg_th_fwhp, tg_F_2 = tele_geo.th_fwhp, tele_geo.F_2

In [4]:
# Perform ray-tracing and simulation
rxmirror = ap.ray_mirror_pts(P_rx, tg_th2, tg_F_2, theta, phi)
apout = ap.aperature_fields_from_panel_model(panel_model1, panel_model2, \
                                    P_rx, tg_th_1, tg_th2, tg_z_ap, tg_th_fwhp, \
                                    theta, phi, rxmirror
                                    )
farbeam = ff.far_field_sim(apout, tele_geo, None)

In [5]:
az = flat_sqgrid_to_ticks(farbeam[0]).real
el = flat_sqgrid_to_ticks(farbeam[1]).real
beam_sky = flat_sqgrid_to_2dgrid(farbeam[2])
beammain = beam_sky[int(tele_geo.N_scan), int(tele_geo.N_scan)]
beam_sky = beam_sky * np.conj(beammain)
beam_sky = beam_sky / np.max(np.abs(beam_sky)) # normalize the field

## Here we plot the phase and amplitude of the sky beam.

In [9]:
############# plot the field on sky #######################################
x = el * 60.0 * 180.0 / np.pi # [arcmin] 
y = az * 60.0 * 180.0 / np.pi # [arcmin]
field = beam_sky
scalar_field_plot(x, y, field, title='Beam on sky', xlabel="el [arcmin]", ylabel="az [arcmin]")

## Here is the graph of the focal plane beam extracted from the simulation data.


In [7]:
# load simulation data 
Npts_x = np.shape(dataset["field_fp"])[0]
Npts_z = np.shape(dataset["field_fp"])[1]
scanrange_x = 120 # [mm]
scanrange_z = 120 # [mm]
P_rx_x = np.linspace(-scanrange_x/2.0, scanrange_x/2.0, Npts_x)
P_rx_y = 209.920654 # mm, position of focus point with rf source located 1km away
P_rx_z = np.linspace(-scanrange_z/2.0, scanrange_z/2.0, Npts_z)

beam_fp = dataset["field_fp"]
beammain = beam_fp[int(Npts_x/2), int(Npts_z/2)]
beam_fp = beam_fp * np.conj(beammain)
beam_fp = beam_fp / np.max(np.abs(beam_fp))

############# plot the field on focal plane #######################################
x = P_rx_z
y = P_rx_x
field = beam_fp
scalar_field_plot(x, y, field, title='Beam on focal plane', xlabel="z [mm]", ylabel="x [mm]")

## We substract and compare the two beams on sky and focal plane, where the sky beam is flipped upside down before the substraction.

In [8]:
# compare beams on sky and focal plane
beam_sky_flipped = np.flipud(beam_sky)
beam_residual = beam_sky_flipped - beam_fp
phase_sky =  np.mod(np.angle(beam_sky_flipped), 2*np.pi)
phase_fp =  np.mod(np.angle(beam_fp), 2*np.pi)
############# plot the beam difference #######################################
x = P_rx_z
y = P_rx_x
phase = np.abs(phase_sky - phase_fp)
amp = np.abs(beam_sky_flipped)/np.abs(beam_fp)

scalar_field_plot(x, y, amp*np.exp(1j*phase), title='Beam difference', xlabel="z [mm]", ylabel="x [mm]", cmap1='IceFire', cmap2="Jet")